In [18]:
import csv
from pathlib import Path
import pandas as pd
import numpy as np

In [19]:
dataset_path = Path("Results")

In [20]:
def process_csv(dataset_path):

    dataframes = {}

    for file_path in dataset_path.glob("*.csv"): #loop through all files in the folder
        #read the csv file and save it as a dataframe in dict
        data =  pd.read_csv(file_path)
        
        dataframe_name = file_path.name[:-4] #remove .csv
        dataframes[dataframe_name] = data
        
    return dataframes

In [21]:
dataframes = process_csv(dataset_path)
dataframes

{'results 1000 len 3':                         Dataset  Total Weight  Cycle Time (s)  Total Time (s)  \
 0       Delorme_1000_NDD_Unit_0           264      243.834752      260.949842   
 1       Delorme_1000_NDD_Unit_1           280      201.481482      220.468326   
 2       Delorme_1000_NDD_Unit_4           277      205.062737      223.441218   
 3       Delorme_1000_NDD_Unit_2           261      211.855260      225.700159   
 4       Delorme_1000_NDD_Unit_3           252      217.764345      233.724684   
 5     Delorme_1000_NDD_Weight_2         17325      208.183654      224.155385   
 6     Delorme_1000_NDD_Weight_1         18562      199.662460      219.849449   
 7     Delorme_1000_NDD_Weight_0         17453      236.578545      256.819895   
 8     Delorme_1000_NDD_Weight_3         17000      221.196317      239.757284   
 9     Delorme_1000_NDD_Weight_4         18397      200.521244      220.559915   
 10    Delorme_1000_NoNDD_Unit_0           253       68.909078       72.6542

In [22]:
dataframes['results 1000 len 3']["Dataset"]

0         Delorme_1000_NDD_Unit_0
1         Delorme_1000_NDD_Unit_1
2         Delorme_1000_NDD_Unit_4
3         Delorme_1000_NDD_Unit_2
4         Delorme_1000_NDD_Unit_3
5       Delorme_1000_NDD_Weight_2
6       Delorme_1000_NDD_Weight_1
7       Delorme_1000_NDD_Weight_0
8       Delorme_1000_NDD_Weight_3
9       Delorme_1000_NDD_Weight_4
10      Delorme_1000_NoNDD_Unit_0
11      Delorme_1000_NoNDD_Unit_1
12      Delorme_1000_NoNDD_Unit_2
13      Delorme_1000_NoNDD_Unit_3
14    Delorme_1000_NoNDD_Weight_0
15      Delorme_1000_NoNDD_Unit_4
16    Delorme_1000_NoNDD_Weight_3
17    Delorme_1000_NoNDD_Weight_2
18    Delorme_1000_NoNDD_Weight_1
19    Delorme_1000_NoNDD_Weight_4
20        Saidman_1000_NDD_Unit_1
Name: Dataset, dtype: object

In [23]:
def combine_results(dataframes_dict):
    #if the Dataset attribute is the same apart from the last character, combine the instances by averaging it per dataframe
    for key, df in dataframes_dict.items():
        df["Dataset Group"] = df["Dataset"].str[:-2] #get rid of the number at the end
        
        
        combined_df = df.groupby("Dataset Group").mean(numeric_only=True)
        combined_df["Run Spec"] = key[8:] #get the run spec from the key (how I saved it originally)
        
        combined_df["Dataset Size"] = combined_df["Run Spec"].str.extract(r"(\d+)").astype(int)
        beta_match = combined_df["Run Spec"].str.extract(r"beta (\d+\.?\d*)") #need some chatgpt magic
        combined_df["Beta"] = beta_match[0].fillna(0).astype(float) #add beta and dataset size to the dataframe

        dataframes_dict[key] = combined_df
        

    
    return dataframes_dict

In [24]:
final = combine_results(dataframes)


C:\Users\shash\AppData\Local\Temp\ipykernel_14528\4239044081.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_df["Beta"] = beta_match[0].fillna(0).astype(float)
C:\Users\shash\AppData\Local\Temp\ipykernel_14528\4239044081.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_df["Beta"] = beta_match[0].fillna(0).astype(float)
C:\Users\shash\AppData\Local\Temp\ipykernel_14528\4239044081.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_o

In [25]:
final

{'results 1000 len 3':                            Total Weight  Cycle Time (s)  Total Time (s)  \
 Dataset Group                                                             
 Delorme_1000_NDD_Unit             266.8      215.999715      232.856846   
 Delorme_1000_NDD_Weight         17747.4      213.228444      232.228386   
 Delorme_1000_NoNDD_Unit           232.0       53.008205       55.594213   
 Delorme_1000_NoNDD_Weight       14623.2       53.457530       56.245201   
 Saidman_1000_NDD_Unit             371.0     2505.916398     2941.475016   
 
                            Total Model Weight  Max Cycle Length    Run Spec  \
 Dataset Group                                                                 
 Delorme_1000_NDD_Unit                   266.8               3.0  1000 len 3   
 Delorme_1000_NDD_Weight               17747.4               3.0  1000 len 3   
 Delorme_1000_NoNDD_Unit                 232.0               3.0  1000 len 3   
 Delorme_1000_NoNDD_Weight             14623

In [26]:
final['results 50 len 3 beta 0.5']

,Total Weight,Cycle Time (s),Total Time (s),Total Model Weight,Max Cycle Length,Run Spec,Dataset Size,Beta
Dataset Group,,,,,,,,
Delorme_50_NDD_Unit,4.4,0.025117,0.082667,7.1,3.0,50 len 3 beta 0.5,50,0.5
Delorme_50_NDD_Weight,255.0,0.038859,0.116765,417.1,3.0,50 len 3 beta 0.5,50,0.5
Delorme_50_NoNDD_Unit,3.0,0.004835,0.082244,4.6,3.0,50 len 3 beta 0.5,50,0.5
Delorme_50_NoNDD_Weight,159.4,0.003349,0.046712,243.5,3.0,50 len 3 beta 0.5,50,0.5
Saidman_50_NDD_Unit,16.0,0.189500,0.375904,20.0,3.0,50 len 3 beta 0.5,50,0.5
Saidman_50_NDD_Weight,1135.0,0.160615,0.342685,1317.8,3.0,50 len 3 beta 0.5,50,0.5
Saidman_50_NoNDD_Unit,12.2,0.091546,0.212355,15.2,3.0,50 len 3 beta 0.5,50,0.5
Saidman_50_NoNDD_Weight,823.4,0.087655,0.212071,977.6,3.0,50 len 3 beta 0.5,50,0.5


In [27]:
#combine all the dataframes into one and save it as a csv
final_df = pd.concat(final.values())
final_df.to_csv("combined_results.csv")